### <i>Hands on Machine Learning with Scikit-Learn and Tensorflow</i>


# Chapter 2 - California House Price Prediction 

(Part 3 of 3)

The objective of this model is to predict house prices given a California census data set.

In [1]:
# housekeeping stuff to set the page width as wide as possible in this notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# import the data from part 2
import os
import pickle
import numpy as np
import pandas as pd

# Need to copy and paste some classes from part 2. There's surely a better way to do this.
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):        # avoid *args and **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
    def fit(self, X, y=None):
        return self      # nothin else to do 
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
# Done with the copy/paste

def load_data():
    if not os.path.isdir('./data') or not os.path.exists('./data/prt2.pkle'):
        raise Exception('No data from part 2 found. Please run part 2 first.')
        os.exit(1)

    with open('./data/prt2.pkle', 'rb') as f:
        data = pickle.load(f)
        
    return data['housing'], data['housing_prepared'], data['housing_labels'], data['full_pipeline'], data['encoder'], data['num_attribs']

housing, housing_prepared, housing_labels, full_pipeline, encoder, num_attribs = load_data()

## Select and Train a Model

Steps so far:

1. Got raw data and explored it
2. Sampled a training and test set
3. Wrote transformation pipelines to clean and prepare the data
4. Finally it's time for Machine Learning Algorithms

Let's start by training a <b>Linear Regression</b> model:

In [3]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Done!! You now have a working Linear Regression model. Let's try it out on a few instances from the training set:

In [4]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:\t", list(lin_reg.predict(some_data_prepared).astype(int)))

Predictions:	 [210644, 317768, 210956, 59218, 189747]


In [5]:

print("Labels:\t\t", list(some_labels.astype(int)))

Labels:		 [286600, 340600, 196900, 46300, 254500]


It works! But we can do better. Let's measure the Root Mean Squared Error (RMSE) on the whole training set using Scikit-Learn's <code>mean_squared_error()</code> function:

In [6]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68628.198198489234

Better than nothing, but not great... most districts median_housing_values range between 120,000 USD and 265,000 USD so a typical prediction error of $68,628 is not very satisfying.

<i>The model is underfitting the data.</i> When this happens it can mean that the features do not provide enough information to make good predictions or that the model is not powerful enough. 

<B> Possible solutions for underfitting</b>:

1. select a more powerful model
2. feed the training algorithm better features
3. reduce the contraints on the model

The model is not regularized so option 3 is not available. You could try to add more features (e.g. the log of the population) but first let's try a more complex model.

Let's traing a <b><code>DecisionTreeRegressor</code></b> (covered in Chapter 6). This is a powerful model capable of finding complex nonlinear relationships in the data:

In [7]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_mse

0.0

Oh shit! Zero error is not a good sign. This model <b>has now badly overfit the data</b>. How can you be sure? As we saw earlier you don't want to touch the test set until you are ready to launch a model you are confident in so you need to use part of the training set for training and part for model validation.

## Better Evaluation Using Cross-Validation

To evaluate the Decision Tree you could use the <code>train_test_split()</code> method to split the training set into a smaller training set and a validation set, then train your models against the smaller training set and evaluate them against the validation set. Would work fine but there is a better alternative. 

Use Scikit-Learn's <i>cross-validation</i> feature. The following code performs <b><i>K-Fold cross-validation</i></b>: it randomly splits the training set into 10 distinct subsets called <i>folds</i>, then it trains and evaluates the Decision Tree model 10 times, picking a different fold for evaluation each time and training on the other 9 folds. The result is an array containing the 10 evaluation scores:

<i>(NOTE: sklearn Cross-Validation expects a utility function (greater is better) rather than a cost function (lower is better) so the scoring function is actually the opposite of the MSE (i.e. a negative value) which is why the code computes -scores befire calculating the square root)</i>


In [8]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels, 
                        scoring='neg_mean_squared_error', cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores\t\t\t:", scores.astype(int))
    print("Mean\t\t\t:", int(scores.mean()))
    print("Standard Deviation\t:", int(scores.std()))

display_scores(tree_rmse_scores)


Scores			: [68654 67610 70981 70563 71847 75512 70146 70308 78232 70341]
Mean			: 71419
Standard Deviation	: 3007


The DecisionTree seems to be worse than the LinearRegression model now!! Notice that the cross-validation allows you to get not only an estimate of the performance of your model but also a measure of how precise this estimate is (i.e. it's Standard Deviation). The Decision Tree has a score of approximately 71,422 +/- 2,696 (re-running the notebook will give different values due to the random selections). You would not have this information if you just used a single validation set. But cross-validation comes at the cost of training the model several times which is not always possible. 

Let's compute the same scores for the Linear Regression model just to be sure:

In [9]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                            scoring='neg_mean_squared_error', cv=10)

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores			: [66782 66960 70347 74739 68031 71193 64969 68281 71552 67665]
Mean			: 69052
Standard Deviation	: 2731


The Decision Tree model is overfitting the data so badly that it performs worse than the Linear Regression model.

Let's try another model: <b><i>RandomForestRegressor</i></b> (detail in Chapter 7). Random Forests work by training many Decision Trees on random subsets of the features, then averaging out their predictions. Building a model on top of many other models is called <i>Ensemble Learning</i> and it is often a great way to push ML algorithms even further:

In [10]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()

forest_reg.fit(housing_prepared, housing_labels)
forest_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, forest_predictions)
forest_rmse = np.sqrt(forest_mse)
print('Forest training RMSE\t:', int(forest_rmse))

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                               scoring='neg_mean_squared_error', cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Forest training RMSE	: 22259
Scores			: [52051 49709 53495 54645 52892 56637 51670 51302 55740 52555]
Mean			: 53070
Standard Deviation	: 2005


Much better!! Random Forests look very promising. But note that the score on the training set is still much lower than on the validation sets meaning that the model is still overfitting the training set. 

<b>Possible solutions for overfitting</b>:

1. simplify the model
2. constrain it (i.e. regularize it)
3. get a lot more training data

However, before diving further into Random Forests you should try out many other models (e.g. Support Vector Machines with different kernels, possibly a neural network, etc) without spending too much time tweaking the hyperparameters. The goal is to shortlist 4 to 5 promising models. 

You should save every model you experiement with so you can come back easily to any model you want. Make sure you also save the hyperparameters and the trained parameters as well as the cross-validation scores and perhaps the actual predictions as well. This will allow you to easily compare scores across model types and compare the types of errors they make. You can save sklearn models using the pickle python module which is efficient at serializing large NumPy arrays.

## Fine Tune Your Model

### Grid Search

Instead of playing with hyperameters manually trying to get a great combination, use Scikit-Learn's <code>GridSearchCV</code> to search for you. You need to tell it which parameters you want to experiment with and what values to try out and it will evaluate all the possible combinations of hyperparameter values using cross-validation. Here's an example using for RandomForestRegressor:

In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [10, 50, 60, 70], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
_ = grid_search.fit(housing_prepared, housing_labels)

# Now pint the best combination of parameters
print(grid_search.best_params_)


{'max_features': 6, 'n_estimators': 70}


The <code>param_grid</code> above tells Scikit-Learn to first evaluate 3x4 = 12 combinactions of n_estimators and max_features (explained in chapter 7) and then try all 2x3 = 6 combinations of hyperameter values in the 2nd dict but this time with the <code>bootstrap</code> hyperparameter set to False (it's default is True).

All in all the grid search will explore 12+6 = 18 combinations of RandomForestRegressor <b>and will train each model 5 times since we're using five-fold cross validation</b>. That means 18x5 = 90 rounds of training.

<i>Note 1: when you have no idea what values to try for a hyperparmeter, a simple approach is to try out consecuituve powers of 10 (or a small number if you want a more fine graned search).</i><br>

You can also get the best estimator directly:

In [12]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=6, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=70, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

And the evaluation scores are available:

In [13]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

55545.5375529 {'max_features': 2, 'n_estimators': 10}
52087.4900497 {'max_features': 2, 'n_estimators': 50}
52376.9516537 {'max_features': 2, 'n_estimators': 60}
52042.3242439 {'max_features': 2, 'n_estimators': 70}
53301.3128484 {'max_features': 4, 'n_estimators': 10}
50047.9825589 {'max_features': 4, 'n_estimators': 50}
49961.5325991 {'max_features': 4, 'n_estimators': 60}
49980.8823779 {'max_features': 4, 'n_estimators': 70}
52286.9818987 {'max_features': 6, 'n_estimators': 10}
49800.2810966 {'max_features': 6, 'n_estimators': 50}
49521.0730646 {'max_features': 6, 'n_estimators': 60}
49288.5571738 {'max_features': 6, 'n_estimators': 70}
52063.3229501 {'max_features': 8, 'n_estimators': 10}
49431.888429 {'max_features': 8, 'n_estimators': 50}
49466.6150953 {'max_features': 8, 'n_estimators': 60}
49563.9499106 {'max_features': 8, 'n_estimators': 70}
62112.6941877 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54612.6605146 {'bootstrap': False, 'max_features': 2, 'n_estimat

<i>Note: if GridSearchCV is initialized with <code>refit=True</code> (the default) then once it finds the best estimator using cross-validation it retrains it on the whole training set. This is usually a good idea since feeding it more data will likely improve its performance.</i>

So the best set up comes out better than the original value from above (~52k USD). 

Congrats, fine tuning over.

<i>Note: don't forget you can treat some of the data preperation steps as hyperparameters. E.g. the grid search will automatically find out whether or not to add a feature you werre not sure about (e.g. the add_bedrooms_per_room hyperparameter of the CombinedAttributesAdder transformer). It may similarly be used to automatically find out the best way to handle outliers, missing features, feature selection, and more.</i>

## Randomized Search

When the hyperparameter search space is large, it might be preferrable to use <code>RandomizedSearchCV</code> instead. It can be used in much the same way as the GridSearchCV class but instead of trying out all possible combinations, it evaluates a given number of random combinations by selecting a random value for each hyperparameter at every iteration. This approach has two main benefits:

1. If you let the randomized search run for say 1000 iterations, this approach will explore 1000 differnt values for each hyperparameter (instead of just the few specified for GridSearchCV)
2. You have more control over the computing budget you want to allocate to hyperparameter search, simply by setting the number of iterations. 


## Ensemble Methods

Another way to fine tune your model is to try to combine the models that perform the best. The group ("ensemble") will often perform better than the best individual model (just like Random Forests perform better than the single Decision Trees they rely on), especially if the individual models make very different types of errors. This is covered in chapter 7.


## Analyze the Best Models and Their Errors

You will often get good insights on the problem by inspecting the best modes. For example the RandomForestRegressor can indicate the relative importance of each attribute for making accurate predictions:

In [14]:
feature_importances = grid_search.best_estimator_.feature_importances_
print(feature_importances)

[  7.67158899e-02   7.37172820e-02   4.22586357e-02   1.82297631e-02
   1.73165559e-02   1.78459116e-02   1.64056552e-02   3.23183269e-01
   6.38103022e-02   1.06913162e-01   7.74596295e-02   1.28646993e-02
   1.44640327e-01   1.00268118e-04   3.41545074e-03   5.12319881e-03]


Let's display these importance scores nest to their corresponding attribute names:

In [15]:
extra_attribs = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_one_hot_attribs = list(encoder.classes_)
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)


[(0.32318326892113713, 'median_income'),
 (0.14464032736975097, 'INLAND'),
 (0.10691316163370912, 'pop_per_hhold'),
 (0.077459629454888171, 'bedrooms_per_room'),
 (0.076715889870797097, 'longitude'),
 (0.073717281968913209, 'latitude'),
 (0.063810302237230401, 'rooms_per_hhold'),
 (0.042258635667847412, 'housing_median_age'),
 (0.018229763119311595, 'total_rooms'),
 (0.01784591160555906, 'population'),
 (0.017316555947187576, 'total_bedrooms'),
 (0.016405655245501027, 'households'),
 (0.012864699294167265, '<1H OCEAN'),
 (0.0051231988069661265, 'NEAR OCEAN'),
 (0.0034154507389063745, 'NEAR BAY'),
 (0.00010026811812751486, 'ISLAND')]

With this information you might want to try dropping some of the less useful features (e.g. apparently only one ocean_proximity category is really useful, so you could try dropping the others).

You should also look at the specific errors that your system makes, then try to understand why it makes them and what could fix the problem (adding extra features or, on the contrary, getting rid of uninformative ones, cleaning up outliers, etc).


## Evaluate Your System on the Test Set

Now it's time to evaluate the final model on the test set. Just get the predictors and the labels from your test set, run your <code>full_pipeline</code> to transform the data (<b>call <code>transform()</code> and *not* <code>fit_transform()</code> !!!</b>) and evaluate the final model on the test set:

In [16]:
# First need to go back to part 1 and get the test set
with open('./data/prt1.pkle', 'rb') as f:
        _, strat_test_set = pickle.load(f)

final_model = grid_search.best_estimator_

X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print(final_rmse)

46906.0261348


The performance will ususally be slightly worse than what you measured using the cross-validation if you did a lot of hyperparameter tuning (because your system ends up fine tuned to perform well on the validation data, and will likely not perform as well on unknown datasets). When this happens you must resist the temptation to tweak the hyperparameters to make the numbers look good on the test set: the improvements would be unlikely to generalize to new data.

To present this for decision makers, document everything, create presentations with clear visualizations and easy-to-remember statements (e.g. "the median income is the number one predictor of housing prices").


## Launch, Monitor and Maintain Your System

Next you need to get your system ready for production. In particular, plugin the production input data sources and write tests.

You also need to write monitoring code to check the live performance at regular intervals and trigger alerts when it drops. This is important to catch not only sudden breakage, but also performance degradataion. This is common because models tend to "rot" as data evolves over time unless models are regularly trained on fresh data.

Evaluating the perforamce will involve sample the systems predictions and evaluating them. This will generally involve human analysis. These analysts may coworkers, field experts or e.g. Amazon's Mechanical Turk or CrowdFlower. Either way, you need to plug the human evaluation into your system.

You should also make sure you evaluate the system's input data quality in case it degrades over time due to a poor quality signal (e.g. a malfunctioning sensor sending random values or another team's output becoming stale) but it may take a while before your montoring triggers an alert. If you monitor the inputs you may catch this earlier. Monitoring the inuts is especially important for online learning systems.

Finally you will generally want to train your models on a regular basis using fresh data. You should automate this process as much as possible. If you don't you might e.g. only update your model every 6 months and your system's performance will fuluctuate severly over time. If your system is an online learning system you should make sure you save snapshots of its state at regular intervals so you can easily roll back to a previously working state.


# Congratulations!!!! 

We're done with Chapter 2.

